In [27]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy as sp
import math
import matplotlib.pyplot as plt
from scipy import signal
import os
setting = pd.read_csv('setting_motion.csv', header = 0, usecols = [0,1])
MOTION = setting.iloc[0,1] #The motion capture data
Label = pd.read_csv(MOTION, header=2, nrows = 1)
Markers = pd.read_csv(MOTION, header=5, )
time = pd.read_csv(MOTION, header=6, usecols = [1])
Markers.describe()

,Frame,Time (Seconds),X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,...,Z.27,X.28,Y.28,Z.28,X.29,Y.29,Z.29,X.30,Y.30,Z.30
count,5983.000000,5983.000000,5983.000000,5983.000000,5983.000000,5983.000000,5983.000000,5983.000000,5983.000000,5983.000000,...,5983.000000,5983.000000,5983.000000,5983.000000,5983.000000,5983.000000,5983.000000,5983.000000,5983.000000,5983.000000
mean,2991.000000,29.910000,0.378349,0.813274,0.273214,0.333419,1.233214,0.233358,0.342677,0.039376,...,0.159843,0.500323,1.093592,0.140340,0.525063,1.514578,0.170880,0.517880,0.819122,0.453392
std,1727.287662,17.272877,0.009238,0.146350,0.041107,0.016949,0.161868,0.041151,0.006395,0.017915,...,0.013362,0.012094,0.171727,0.026536,0.026972,0.176119,0.121103,0.012296,0.156972,0.042828
min,0.000000,0.000000,0.319509,0.351483,0.161141,0.241351,0.700538,0.071688,0.321743,0.027977,...,0.112670,0.439007,0.526863,0.082407,0.470790,0.971462,-0.216105,0.442344,0.305590,0.341397
25%,1495.500000,14.955000,0.372915,0.682387,0.247198,0.326434,1.093530,0.214293,0.338277,0.035890,...,0.158995,0.493459,0.947676,0.123639,0.511702,1.362928,0.136358,0.510836,0.678511,0.425542
50%,2991.000000,29.910000,0.377887,0.879257,0.274711,0.332738,1.289363,0.233549,0.341596,0.037854,...,0.159816,0.498949,1.144745,0.137948,0.521506,1.552699,0.210134,0.517646,0.894612,0.453653
75%,4486.500000,44.865000,0.384432,0.939138,0.297069,0.339663,1.377231,0.256134,0.347380,0.039618,...,0.160517,0.507356,1.250226,0.150073,0.532972,1.688864,0.253440,0.525853,0.953002,0.479874
max,5982.000000,59.820000,0.401822,1.200349,0.374871,0.411004,1.654556,0.397411,0.364541,0.309050,...,0.269304,0.542162,1.533056,0.283203,0.670410,1.923753,0.336723,0.550973,1.209892,0.567061


### Getting the label for converting the data

In [28]:
Label_1= Label.columns.values
Markers_Label = []

#Need to check the number of marker used in the experiment
#
Marker_number = 31
for i in range(0,Marker_number):
    Temp = Label_1[2+3*i]
    Temp_1 = Temp.replace("Minh:", "") #Need to remove the Marker set name
    Markers_Label.append(Temp_1)
Markers_Label

['L.ASIS',
 'L.Acromium',
 'L.Heel',
 'L.Midfoot.Lat',
 'L.Midfoot.Sup',
 'L.Shank.Front',
 'L.Shank.Rear',
 'L.Shank.Upper',
 'L.Thigh.Front',
 'L.Thigh.Rear',
 'L.Thigh.Upper',
 'L.Toe.Lat',
 'L.Toe.Med',
 'L.Toe.Tip',
 'R.ASIS',
 'R.Acromium',
 'R.Heel',
 'R.Midfoot.Lat',
 'R.Midfoot.Sup',
 'R.Shank.Front',
 'R.Shank.Rear',
 'R.Shank.Upper',
 'R.Thigh.Front',
 'R.Thigh.Rear',
 'R.Thigh.Upper',
 'R.Toe.Lat',
 'R.Toe.Med',
 'R.Toe.Tip',
 'Sternum',
 'Top.Head',
 'V.Sacral']

In [29]:
output_name = setting.iloc[2,1]
output_path = setting.iloc[1,1]
if os.path.exists(output_path + output_name + '.trc'):
    os.remove(output_path + output_name + '.trc')
Markers.to_csv(output_path + output_name + '.trc',  sep='\t', index=False, header = False)

In [31]:
New_label_Coor = '\t'
New_label_Mar = 'Frame#\tTime'
Num_frames = '5983'
Num_markers = '31' 

for i in range(0,Marker_number):
    New_label_Coor = New_label_Coor + '\t' + 'X' + str(i+1) +'\t' + 'Y' + str(i+1) + '\t' + 'Z' + str(i+1)
for i in range(0,Marker_number-1):
    New_label_Mar = New_label_Mar + '\t' + Markers_Label[i] + '\t\t'
New_label_Mar = New_label_Mar + '\t' + Markers_Label[Marker_number-1] + '\n'

In [33]:
Contents = 'PathFileType\t4\t' + '(X,Y,Z)\t' + output_name + '.trc' + '\n' \
            + 'DataRate\tCameraRate\tNumFrames\tNumMarkers\tUnits\tOrigDataRate\tOrigDataStartFrame\tOrigNumFrames' + '\n' \
            + '100\t100\t' + Num_frames + '\t' + Num_markers + '\t' + 'm\t100\t1\t' + Num_frames + '\n'

In [34]:
with open(output_path + output_name + '.trc', "r+") as f:
    old = f.read() # read everything in the file
    f.seek(0) # rewind
    f.write(Contents + New_label_Mar + New_label_Coor + '\n\n' + old) #